## II. Phase equilibrium

In this section we will study the equilibrium between water (or a solution) and a gas (mainly CO2). Then, we will focus on the equilibrium between one or more minerals and water, and finally, we will examine the equilibrium of water with both a gas and a mineral.

All these combinations are built from a template very similar to the one studied in "Speciating a natural water," but using the keyword EQUILIBRIUM_PHASES.

In this section uisng some of the examples we will solve at least the following geochemical problems:

- pH of rain.
- Mineral solubility.
- Relative humidity of deliquescence.
- Solubility of carbonates in the presence of CO2.
- Stability of minerals with different degrees of hydration (e.g., gypsum-anhydrite) and polymorphic minerals (e.g., calcite-aragonite).
- Open system vs. closed system with respect to CO2.

### 1. Water-CO$_2$ Equilibrium

There are many situations in geochemistry where CO2 plays an important role, such as the pH of rainwater, the formation of speleothems in karst systems, etc. Although we will focus on CO2, this template can also be used with other gases, such as O2.

The template we will use to study the equilibrium between water or a solution and CO2 has the following structure:

```
TITLE Plantilla: Equilibrio entre el agua y el CO2.
SOLUTION 1
    temp      20
    pe        4
    pH        7
    -water    1     # kg
EQUILIBRIUM_PHASES
    CO2(g)    -3.50
END
```

This input file performs the speciation of water (calculates the distribution of species) in equilibrium with CO2. Therefore, this template has two parts:

In the first part, the characteristics of the water are defined under the SOLUTION keyword. This example uses pure water, with characteristics of temperature equal to 25 ºC, pH = 7, and pe = 4. We could also use a solution like the one used in the "Speciating a natural water" template.
In the second part, the partial pressure of CO2 in equilibrium with the water is defined using the EQUILIBRIUM_PHASES keyword. The partial pressure is entered as its logarithm, i.e., log pCO2 (-3.5 = log10⁻³.⁵ atm).
The structure of this template implies that the CO2 pressure is always constant: we are considering an open system.

The output file shows the two parts of the input file. First, it displays the speciation of pure water (Beginning of initial solution calculations), followed by the speciation considering equilibrium with CO2 (Beginning of batch-reaction calculations). It is interesting to study the results of the second part, where CO2 defines the characteristics of the final solution. Notice the slightly acidic pH value of the resulting water (in the Description of solution), and that the CO2 has the same partial pressure as the initial one, meaning the partial pressure is constant (see Saturation indices).

In [ ]:
import subprocess

# Step 1: Create the PHREEQC input file
input_file_content = """
TITLE Plantilla: Equilibrio entre el agua y el CO2.
SOLUTION 1
temp      20
pe        4
pH        7
-water    1       # kg
EQUILIBRIUM_PHASES
CO2(g)          -3.50  
END
"""

# Save the input file
input_file_name = "phreeqc_example2.pqi"
with open(input_file_name, "w") as file:
    file.write(input_file_content)
print(f"PHREEQC input file '{input_file_name}' created successfully.")

# Step 2: Run PHREEQC using subprocess
output_file_name = "phreeqc_example2_out.txt"
database_file = "/srv/data/phreeqc-3.7.3-15968/database/phreeqc.dat"  # Update the path if necessary
phreeqc_executable = "/srv/data/phreeqc-3.7.3-15968/src/phreeqc"  # Use "phreeqc.exe" on Windows, or the full path to the executable

# Run PHREEQC
try:
    subprocess.run([phreeqc_executable, input_file_name, output_file_name, database_file], check=True)
    print(f"PHREEQC run completed. Output saved in '{output_file_name}'.")
except subprocess.CalledProcessError as e:
    print(f"PHREEQC execution failed: {e}")    

# Display the contents of the output file, ignoring problematic characters
try:
    with open(output_file_name, "r", encoding="utf-8", errors="ignore") as output_file:
        output_content = output_file.read()
    print("PHREEQC Output:\n")
    print(output_content)
except FileNotFoundError:
    print(f"Output file '{output_file_name}' not found.")

## Problem 1 
Calculate the pH of rain water at different P$_{CO_2}$ and a temperature of 10, 30, 50 centigrades. What variable between pH and temperature plays a mayor role on the pH?   

### 2. Water-mineral equilibrium

#### 2.1 Water - Halite

The structure of this template is very similar to the previous one and has the following (abbreviated) format:


SOLUTION 1; temp 25; EQUILIBRIUM_PHASES; Halite 0 10; END

In this example, we bring pure water into contact with halite (NaCl) until equilibrium is reached. Unlike CO2, when working with a mineral, the saturation index of the mineral and the number of moles used for the calculation must be defined. That is, Halite 0 10 corresponds to the mineral in contact with the water (halite), the saturation index (0, which corresponds to equilibrium), and the number of moles used for the calculation, which is 10. It is recommended to use a high number of moles to avoid calculation problems. PHREEQC uses 10 moles by default. If the mineral has zero moles in equilibrium with the water, then the mineral cannot dissolve but can precipitate.

In this example, a saturation index of 0 is used, meaning the mineral is in equilibrium with the solution. However, we can define the value we consider appropriate for our calculations. For instance, if the saturation index is 1, the solution would be oversaturated, while if it is -0.5, the solution would be undersaturated.

The name of the mineral is written in English. A quick way to check the correct spelling of minerals is to look it up in the PHREEQC Database sheet under the PHASES section.

This type of template is very useful for calculating the solubility of a mineral in water at different temperatures, quantifying the variation in its solubility due to the presence of other ions (natural water), and calculating its relative humidity of deliquescence.

The output file, as in the previous case of water in equilibrium with CO2, shows the two parts of the input file. First, it displays the speciation of pure water, followed by the speciation considering equilibrium with halite (Beginning of batch-reaction calculations). The second part (Solution composition) calculates the moles of Na and Cl that satisfy an SI=0. In other words, we are obtaining the value of halite solubility. In the Description of solution, several interesting parameters are obtained, such as pH, electrical conductivity, density, ionic strength, and water activity. At equilibrium, the chemical potential of the liquid water is equal to the chemical potential of the water vapor, so the water activity is equal to the relative humidity at equilibrium or the relative humidity of deliquescence, expressed as a fraction.

Previously, we mentioned that the equilibrium between pure water and halite is expressed as halite 0 10. In this situation, PHREEQC dissolves (removes ions from the solid) or precipitates (adds ions to the solid) until equilibrium is reached with a given water. This variation is observed in the Beginning of batch-reaction calculations; a negative sign indicates that the mineral is dissolving until equilibrium is reached, while a positive sign implies mineral precipitation.

If our simulation needs to involve only precipitation or only dissolution, then we must indicate it as follows:

- halite 0 0 (precipitates but does not dissolve the mineral)
- halite 0 10 dis (dissolves but does not precipitate the mineral)

In [ ]:
import subprocess

# Step 1: Create the PHREEQC input file
input_file_content = """
TITLE Plantilla: Equilibrio entre el agua y el CO2.
SOLUTION 1
temp      25
EQUILIBRIUM_PHASES
Halite 0 10            
END
"""

# Save the input file
input_file_name = "phreeqc_example3.pqi"
with open(input_file_name, "w") as file:
    file.write(input_file_content)
print(f"PHREEQC input file '{input_file_name}' created successfully.")

# Step 2: Run PHREEQC using subprocess
output_file_name = "phreeqc_example3_out.txt"
database_file = "/srv/data/phreeqc-3.7.3-15968/database/phreeqc.dat"  # Update the path if necessary
phreeqc_executable = "/srv/data/phreeqc-3.7.3-15968/src/phreeqc"  # Use "phreeqc.exe" on Windows, or the full path to the executable

# Run PHREEQC
try:
    subprocess.run([phreeqc_executable, input_file_name, output_file_name, database_file], check=True)
    print(f"PHREEQC run completed. Output saved in '{output_file_name}'.")
except subprocess.CalledProcessError as e:
    print(f"PHREEQC execution failed: {e}")    

# Display the contents of the output file, ignoring problematic characters
try:
    with open(output_file_name, "r", encoding="utf-8", errors="ignore") as output_file:
        output_content = output_file.read()
    print("PHREEQC Output:\n")
    print(output_content)
except FileNotFoundError:
    print(f"Output file '{output_file_name}' not found.")##

In [ ]:
import subprocess

# Step 1: Create the PHREEQC input file
input_file_content = """
TITLE Plantilla: Equilibrio entre el agua y el CO2.
SOLUTION 1
temp      25
EQUILIBRIUM_PHASES
Halite 0 10            
END
"""

# Save the input file
input_file_name = "phreeqc_example3.pqi"
with open(input_file_name, "w") as file:
    file.write(input_file_content)
print(f"PHREEQC input file '{input_file_name}' created successfully.")

# Step 2: Run PHREEQC using subprocess
output_file_name = "phreeqc_example3_out.txt"
database_file = "/srv/data/phreeqc-3.7.3-15968/database/phreeqc.dat"  # Update the path if necessary
phreeqc_executable = "/srv/data/phreeqc-3.7.3-15968/src/phreeqc"  # Use "phreeqc.exe" on Windows, or the full path to the executable

# Run PHREEQC
try:
    subprocess.run([phreeqc_executable, input_file_name, output_file_name, database_file], check=True)
    print(f"PHREEQC run completed. Output saved in '{output_file_name}'.")
except subprocess.CalledProcessError as e:
    print(f"PHREEQC execution failed: {e}")    

# Display the contents of the output file, ignoring problematic characters
try:
    with open(output_file_name, "r", encoding="utf-8", errors="ignore") as output_file:
        output_content = output_file.read()
    print("PHREEQC Output:\n")
    print(output_content)
except FileNotFoundError:
    print(f"Output file '{output_file_name}' not found.")##

#### 2.2. Equilibrium Water and Calcite

The following PHREEQC exercise and is designed to further familiarize you with the program’s input and output by calculating the concentration of Ca and CO3 in a solution that is in equilibrium with calcite. The first simulation produces a system where calcite and pure water are in equilibrium (input file 1a). As you look at the output you should note that the pH changes from an initial value of 7 to about 9.9 and that additional dissolved species are formed besides Ca and CO3 ions.

The input for this process is the following:

```
TITLE calcite solubility 
SOLUTION 1 pure water & calcite
pH 7
temp 25
EQUILIBRIUM_PHASES
Calcite 0.0 1
END
```

here a mole of calcite reacts with a pure water solution to reach saturated conditions. 

In [ ]:
import subprocess

# Step 1: Create the PHREEQC input file
input_file_content = """
TITLE calcite solubility 
SOLUTION 1 pure water & calcite
pH 7
temp 25
EQUILIBRIUM_PHASES
Calcite 0.0 1
END
"""

# Save the input file
input_file_name = "phreeqc_example4.pqi"
with open(input_file_name, "w") as file:
    file.write(input_file_content)
print(f"PHREEQC input file '{input_file_name}' created successfully.")

# Step 2: Run PHREEQC using subprocess
output_file_name = "phreeqc_example4_out.txt"
database_file = "/srv/data/phreeqc-3.7.3-15968/database/phreeqc.dat"  # Update the path if necessary
phreeqc_executable = "/srv/data/phreeqc-3.7.3-15968/src/phreeqc"  # Use "phreeqc.exe" on Windows, or the full path to the executable

# Run PHREEQC
try:
    subprocess.run([phreeqc_executable, input_file_name, output_file_name, database_file], check=True)
    print(f"PHREEQC run completed. Output saved in '{output_file_name}'.")
except subprocess.CalledProcessError as e:
    print(f"PHREEQC execution failed: {e}")    

# Display the contents of the output file, ignoring problematic characters
try:
    with open(output_file_name, "r", encoding="utf-8", errors="ignore") as output_file:
        output_content = output_file.read()
    print("PHREEQC Output:\n")
    print(output_content)
except FileNotFoundError:
    print(f"Output file '{output_file_name}' not found.")##

## Problem 2 

### Concepts of Aqueous Speciation and Solubility of Solids and Gases

We have calculated the concentration of Ca and CO3 in a solution in equilibrium with calcite at 25°C and a final pH of 9.9. However, after examining the output we can see that there are other species in solution that are related to the simple equation we thought we were modeling. The interactions between water (H2O) and dissolved calcite produced several other aqueous forms of Ca and CO3 (species). The CO3²⁻ is related to HCO3⁻ and H2CO3 by a series of equilibrium equations. The total of dissolved carbonate species is referred to as "carbonate alkalinity". A complete description of the system requires us to include additional chemical equations:

CO2(g) ↔ CO2(aq)

CO2(aq) + H2O ↔ H2CO3

H2CO3 ↔ H⁺ + HCO3⁻

HCO3⁻ ↔ H⁺ + CO3²⁻

The equilibrium constants for these reactions are usually labeled KCO2, K1, and K2, respectively. The reaction of CO2(aq) to form H2CO3 is usually combined with the dissolution of CO2(g) to form CO2(aq) reaction and the sum of the reactions has the constant KCO2. These equations, together with the dissolution of calcite equation if calcite is present, can be used to describe the carbonate system.

The alkalinity value reported for water samples is determined from a titration of the water sample with H2SO4, and is reported as meq/l or eq/l CO3, meq/l or eq/l HCO3 or meq/l or eq/l CaCO3. Alkalinity is actually equal to total HCO3⁻ + CO3²⁻ + any other weak acids that are negatively charged at natural water pH values of 7 to 10 (usually boron and organic acids). This value is calculated by measuring the number of equivalents of acid needed to change the water sample (of known volume) from its starting pH to a pH of 4.5. The factor of H⁺ itself is used as eq/l of alkalinity in the water. Sometimes this data is further converted to eq/l or meq/l of HCO3 and/or CO3 using the pH of the water and assuming that the ratio of HCO3/CO3 is a function of pH (10⁻¹0.3 = CO3²⁻ * H⁺/HCO3⁻).

Using the information above and taking into account that a closed system is such that is not in equilibrium with atmospheric CO2 as is an open system, which system dissolves more calcite? To do this, obtain the concentration of Ca and HCO3, as well as the pH and the partial pressure of CO2 for solutions in equilibrium with calcite and CO2. 